In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
# Load the dataset
df = pd.read_csv('/content/IMDb_Movies_India.csv', encoding='ISO-8859-1')
df.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [58]:
# Step 1: Check a sample of 'Duration' column to understand its format
print(df['Duration'].sample(10))

# Step 2: Ensure 'Duration' column is treated as strings
df['Duration'] = df['Duration'].astype(str)

# Step 3: Clean 'Duration' values by removing non-numeric characters (like commas)
df['Duration'] = df['Duration'].str.replace(',', '')

# Step 4: Convert cleaned 'Duration' values to float
df['Duration'] = pd.to_numeric(df['Duration'], errors='coerce')

# Step 5: Fill missing values in 'Duration' with the median
df['Duration'].fillna(df['Duration'].median(), inplace=True)

# Check if there are any remaining missing values in the dataframe
print(df.isnull().sum())

# Optionally, verify the data type of 'Duration'
print(df['Duration'].dtype)

14307    138 min
3204      92 min
5678         NaN
9154         NaN
3033     112 min
809          NaN
1967         NaN
14931    128 min
3740     150 min
5581     136 min
Name: Duration, dtype: object
Name            0
Year          528
Duration    15509
Genre        1877
Rating       7590
Votes        7589
Director      525
Actor 1      1617
Actor 2      2384
Actor 3      3144
dtype: int64
float64


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


In [59]:
# Encode categorical variables using one-hot encoding
df = pd.get_dummies(df, columns=['Genre', 'Director', 'Actor 1', 'Actor 2', 'Actor 3'], drop_first=True)

# Verify the dataframe
df.head()

,Name,Year,Duration,Rating,Votes,"Genre_Action, Adventure","Genre_Action, Adventure, Biography","Genre_Action, Adventure, Comedy","Genre_Action, Adventure, Crime","Genre_Action, Adventure, Drama",...,Actor 3_Zeenat Aman,Actor 3_Zeeshan Khan,Actor 3_Zeishan Quadri,Actor 3_Zenobia Shroff,Actor 3_Zohra,Actor 3_Zoya Hussain,Actor 3_Zubeida,Actor 3_Zuber K. Khan,Actor 3_Zulfi Sayed,Actor 3_Zunaid Memon
0,,NaN,NaN,NaN,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,#Gadhvi (He thought he was Gandhi),(2019),NaN,7.0,8,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,#Homecoming,(2021),NaN,NaN,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,#Yaaram,(2019),NaN,4.4,35,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,...And Once Again,(2010),NaN,NaN,NaN,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [60]:
# Define features and target
X = df.drop('Rating', axis=1)
y = df['Rating']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [66]:
# Impute missing values in the target variable
y_train = y_train.fillna(y_train.median())

# Ensure 'Year', 'Duration', 'Votes' are numeric, handle non-numeric values and commas
numeric_features = ['Year', 'Duration', 'Votes']
for feature in numeric_features:
    X_train[feature] = pd.to_numeric(X_train[feature].astype(str).str.replace(',', ''), errors='coerce')

# Identify categorical features
categorical_features = X_train.select_dtypes(include=['object']).columns

# Define transformers for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append regression model to preprocessing pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# Train the model
model.fit(X_train, y_train) # Now the model should fit without the ValueError

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Year', 'Duration',
                                                   'Votes']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Name'], dtype='object'))])),
                ('regressor', LinearRegression())])

In [78]:
try:
    # Define features and target
    X = df.drop('Rating', axis=1)
    y = df['Rating']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print("Mean Squared Error:", mse)
    print("R-squared:", r2)

except Exception as e:
    print(f"Error occurred: {e}")

Error occurred: Cannot use median strategy with non-numeric data:
could not convert string to float: '(1996)'
